# `06-rag`

# RAG(검색-증강 생성) Retrieval-Augmented Generation

- LLM이 외부 데이터를 컨텍스트로 활용.

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [8]:
%pip install -q "langchain_chroma>=0.1.2" langchain_community faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# OpenAI 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

# VectorStore 에 임베딩 후 저장 (in memory)
vectorstore = Chroma.from_documents(
    documents,
    embedding=embedding
)

In [20]:
vectorstore.similarity_search(
    '초보자', k=2
)

[Document(id='05ddbfb0-2154-4e5f-b145-3e8dec2fad8f', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(id='47c2635c-5b93-448b-8a57-727f3edd6ed5', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

## 사전처리 단계
1. 문서 불러오기 (Loading)
2. 텍스트 나누기 (Splitting)
3. 숫자로 바꾸기 (Embedding)
4. 저장하기 (VectorStore)

In [15]:
%pip install -q pymupdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()

print(len(docs))

23


In [18]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 500 글자 당 1 청크 / 50 글자는 겹치게 나눈다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

print(len(split_docs))

72


In [23]:
# 3. Vector Embedding + Vector Store
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)

# Test
vectorstore.similarity_search(
    '미국 대통령과 관련된 문서들 가져와봐', k=4
)

[Document(id='ec9f21d3-adb4-44ad-9824-f278f3fed74d', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 3}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한 행정 조치를 명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\nKEY Contents\n£ 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄'),
 Docu

## 검색 증강 단계
1. 사용자 질문 (Query)
2. 검색 (Retrieve)
3. LLM
4. 저장하기 (VectorStore)

In [26]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain import hub

# Prompt 세팅
prompt = hub.pull('rlm/rag-prompt')

# LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성(retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('삼성전자 관련 소식 다 가져와')

"삼성전자는 2023년 11월 8일 '삼성 AI 포럼 2023'에서 자체 개발한 생성 AI '삼성 가우스'를 공개했으며, 언어, 코드, 이미지 모델로 구성되어 온디바이스에서 작동 가능합니다. 삼성 가우스는 안전한 데이터 학습과 다양한 제품에 단계적으로 탑재될 예정이며, 2024년에는 경쟁 제품들과 경쟁할 것으로 예상됩니다. 자세한 내용은 공식 홈페이지(spri.kr) 또는 AI 정책연구실 문의처를 참고하시기 바랍니다."

In [28]:
# Agent + RAG
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_tavily import TavilySearch
from datetime import datetime
# RAG 관련
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

today = datetime.today().strftime('%Y-%m-%d')

llm = ChatOpenAI(model='gpt-4.1-nano')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding=embedding)
retriever = vectorstore.as_retriever()

rag_tool = create_retriever_tool(
    retriever,
    name='pdf_search',
    description='PDF 문서에서 질문과 관련된 내용을 검색합니다.'  # Agent가 언제 이 tool을 쓸지 알게됨
)

text = f"""
너는 웹 검색이 가능하고, 2023년 12월 인공지능 산업 최신동향 정보를 담은 pdf 를 검색할 수 있는 어시스턴트야.

- 사용자가 PDF 문서와 관련된 질문(ex. '이 pdf에서', '문서내용', '파일에서)을 하면 반드시 'pdf_search' 도구를 써야해
- 사용자 질문이 팩트체크를 필요로 하고, 최신성이 필요하다 판단되면 web_search를 실행해야해
- 사용자가 일반적인 질문을 하고, 최신성이나 팩트체크가 필요없으면 그냥 답변해
- 뭔가 확실하지 않으면 pdf_search 와 web_search를 모두 실행해서 답변을 생성해

오늘은 {today} 야.
"""


prompt = ChatPromptTemplate.from_messages([
    ('system', text),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool, rag_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    memory=memory, 
    tools=[search_tool, rag_tool],
    verbose=True)

In [29]:
agent_executor.invoke({'input': '방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?'})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '2023년 이후 인공지능 산업 최신 동향', 'search_depth': 'advanced', 'time_range': 'year'}`


{'query': '2023년 이후 인공지능 산업 최신 동향', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://ettrends.etri.re.kr/ettrends/212/0905212008/', 'title': '인공지능 분야 산업·기술 동향 및 이슈', 'content': 'D. Pinto, "NVIDIA Announces Project GR00T Foundation Model for Humanoid Robots and Major Isaac Robotics Platform Update," NVIDIA Newsroon, Mar. 18, 2024. \n\nSamsung, "온디바이스AI." \n\n임양섭, "멀티모달 AI 경쟁과 다가오는 AGI," IITPICT SPOT ISSUE ICT 산업동향, SPOT 2024-02, 2024, pp. 1-21.\n\nM.R. Morris et al., "Levels of AGI for Operationalizing Progress on the Path to AGI," Proceedings of ICML, 2024. \n\nForbes, "OpenAI’s 5 Levels of ‘Super AI’(AGI To Outperform Human Capability)," July 16, 2024. [...] Marketsandmarkets, "Artificial Intelligence (AI) Market," May. 2024.\n\nGartner, "Gartner Top 10 Strategic Technology Trends for 20

{'input': '방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?',
 'chat_history': [HumanMessage(content='방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2023년 이후 인공지능 산업은 더욱 빠른 성장과 혁신을 보이고 있습니다. 최근 조사에 따르면, 전 세계 인공지능 시장 규모는 2023년 기준으로 2,146억 달러에 달하며, 2030년에는 1조 3,391억 달러까지 성장할 전망입니다(연평균 35.7%). 또한, 인공지능 기술의 발전과 함께 생성형 AI, AI 에이전트, 멀티모달 모델 등 다양한 신기술이 상용화되고 있습니다.\n\n특히, 글로벌 투자가 다시 활발해지고 있으며, 미국과 중국의 경쟁이 치열한 가운데, AI의 성능은 매년 급격히 향상되고 있습니다. 기업들은 고객 서비스, 마케팅, 제조업 등 다양한 산업 분야에서 AI를 적극 활용하고 있으며, 국내 시장도 2024년에는 약 6.3조 원 규모로 안정기에 접어들어 성장세를 유지하고 있습니다.\n\n이밖에도 AI 규제 법률 강화, 책임있는 AI 개발 노력, 특허 출원 증가 등 산업 전반에 걸친 변화가 지속되고 있음을 볼 수 있습니다. 2024년과 2025년에 걸쳐 산업의 다양한 분야에서 AI의 역할이 더욱 확대되고 있으며, 산업별 맞춤형 모델과 고도화된 AI 솔루션이 등장할 것으로 기대되고 있습니다.', additional_kwargs={}, response_metadata={})],
 'output': '2023년 이후 인공지능 산업은 더욱 빠른 성장과 혁신을 보이고 있습니다. 최근 조사에 따르면, 전 세계 인공지능 시장 규모는 2023년 기준으로 2,146억 달러에 달하며, 2030년에는 1조 3,391억 달러까지 성장할 전망입니다(연평균 35.7%). 또한, 인공지능 기술의 발전과 함께 생성형 AI, AI 에이전트, 멀티